In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import KFold
import warnings
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.max_columns', 500)
from scipy.stats import mode
from scipy import stats
import feather
import re
import warnings
import random

from IPython.display import clear_output

In [2]:
all_transactions = feather.read_dataframe('all_transactions')

In [3]:
all_transactions['no_installments'] = (all_transactions.installments == 0).astype(np.int16)
all_transactions['NNN_installments'] = (all_transactions.installments == 999).astype(np.int16)
all_transactions.installments.replace({999:np.nan},inplace=True)

In [4]:
all_transactions.head() 

,authorized_flag,card_id,city_id,category_1,installments,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,state_id,subsector_id,no_city,undefined_installments,big_installments,cat2_dum_0.0,cat2_dum_1.0,cat2_dum_2.0,cat2_dum_3.0,cat2_dum_4.0,cat2_dum_5.0,cat2_dum_nan,cat3_dum_A,cat3_dum_B,cat3_dum_C,cat3_dum_nan,purchase_amount_unusual,purchase_date_year,purchase_date_season,purchase_date_year_quarter,purchase_date_activity_month,purchase_date_month,purchase_date_week,purchase_date_all_weeks,purchase_date_months_week,purchase_date_end_of_the_month,purchase_date_start_of_the_month,purchase_years_day,purchase_date_day,purchase_date_weekend,purchase_date_hour,purchase_date_day_cycle,payment_per_month,position_from_start_to_finnish,month_lag_min,number_of_purchases,payment_from_end_to_begging,payment_from_begging_to_end,valid_authorization_percent,purchase_date_weekday,no_installments,NNN_installments
0,1,C_ID_4e6213e9bc,11,0,0.0,10,M_ID_e020e9b302,-10,22.966691,2017-06-25 15:33:07,6,12,0,0,0,0,1,0,0,0,0,0,1,0,0,0,False,0,2,2,6,6,25,25,3,0,0,176,176,1,15,1,0.0,6,-15,402,10,5,0.977612,6,1,0
1,1,C_ID_4e6213e9bc,11,0,0.0,10,M_ID_86ec983688,-9,22.668715,2017-07-15 12:10:45,6,10,0,0,0,0,1,0,0,0,0,0,1,0,0,0,False,0,3,3,7,7,28,28,2,0,0,196,196,0,12,1,0.0,7,-15,402,9,6,0.977612,5,1,0
2,1,C_ID_4e6213e9bc,11,0,0.0,10,M_ID_979ed661fc,-8,22.796140,2017-08-09 22:04:29,6,12,0,0,0,0,1,0,0,0,0,0,1,0,0,0,False,0,3,3,8,8,32,32,1,0,0,221,221,0,22,0,0.0,8,-15,402,8,7,0.977612,2,1,0
3,1,C_ID_4e6213e9bc,11,0,0.0,8,M_ID_e6d5ae8ea6,-7,22.646476,2017-09-02 10:06:26,6,11,0,0,0,0,1,0,0,0,0,0,1,0,0,0,False,0,3,3,9,9,35,35,0,0,0,245,245,0,10,1,0.0,9,-15,402,7,8,0.977612,5,1,0
4,1,C_ID_4e6213e9bc,11,0,0.0,10,M_ID_e020e9b302,-13,22.771346,2017-03-10 01:14:19,6,12,0,0,0,0,1,0,0,0,0,0,1,0,0,0,False,0,1,1,3,3,10,10,1,0,0,69,69,0,1,0,0.0,3,-15,402,13,2,0.977612,4,1,0


In [5]:
last_month = all_transactions[all_transactions.payment_from_end_to_begging == 0]

In [6]:
other_months = all_transactions[all_transactions.payment_from_end_to_begging != 0]

In [10]:
def diff_in_days_max(x):
    value = (x.sort_values().diff()).max()
    if np.isnan(value):
        return 0
    else: return value

def diff_in_days_min(x):
    value = (x.sort_values().diff()).min()
    if np.isnan(value):
        return 0
    else: return value

def diff_in_days_mean(x):
    value = (x.sort_values().diff()).mean()
    if np.isnan(value):
        return 0
    else: return value
    
def diff_in_days_std(x):
    value = (x.sort_values().diff()).std()
    if np.isnan(value):
        return 0
    else: return value
    
def diff_in_days_std(x):
    value = (x.sort_values().diff()).sum()
    if np.isnan(value):
        return 0
    else: return value

In [11]:
last_month_groupped = last_month.groupby(['card_id']).agg({'purchase_amount':['max','mean','min','std','sum'],
                                                                                        'authorized_flag':'mean',
                                                                                        'installments':['max','mean','min','std','sum'],
                                                                                        'no_installments':'mean',
                                                                                        'NNN_installments':'mean',
                                                                                        'no_city':'mean',
                                                                                        'cat2_dum_0.0':'mean',
                                                                                        'cat2_dum_1.0':'mean',
                                                                                        'cat2_dum_2.0':'mean',
                                                                                        'cat2_dum_3.0':'mean',
                                                                                        'cat2_dum_4.0':'mean',
                                                                                        'cat2_dum_5.0':'mean',
                                                                                        'cat3_dum_A':'mean',
                                                                                        'cat3_dum_B':'mean',
                                                                                        'cat3_dum_C':'mean',
                                                                                        'cat3_dum_nan':'mean',
                                                                                        'purchase_amount_unusual':'mean',
                                                                                        'purchase_date_day':[diff_in_days_max,diff_in_days_min,diff_in_days_mean,diff_in_days_std]})

In [12]:
last_month_groupped.reset_index(inplace=True)

In [13]:
 last_month_groupped.columns = [f'last_month_{x[0]}_{x[1]}' if x[0]!='card_id' else x[0] for x in last_month_groupped]

In [14]:
other_months_groupped = other_months.groupby(['card_id','payment_from_end_to_begging']).agg({'purchase_amount':['max','mean','min','std','sum'],
                                                                                        'authorized_flag':'mean',
                                                                                        'installments':['max','mean','min','std','sum'],
                                                                                        'no_installments':'mean',
                                                                                        'NNN_installments':'mean',
                                                                                        'no_city':'mean',
                                                                                        'cat2_dum_0.0':'mean',
                                                                                        'cat2_dum_1.0':'mean',
                                                                                        'cat2_dum_2.0':'mean',
                                                                                        'cat2_dum_3.0':'mean',
                                                                                        'cat2_dum_4.0':'mean',
                                                                                        'cat2_dum_5.0':'mean',
                                                                                        'cat3_dum_A':'mean',
                                                                                        'cat3_dum_B':'mean',
                                                                                        'cat3_dum_C':'mean',
                                                                                        'cat3_dum_nan':'mean',
                                                                                        'purchase_amount_unusual':'mean',
                                                                                        'purchase_date_day':[diff_in_days_max,diff_in_days_min,diff_in_days_mean,diff_in_days_std]})

In [15]:
other_months_groupped.reset_index(inplace=True)

In [16]:
 other_months_groupped.columns = [f'other_months_{x[0]}_{x[1]}' if x[0]!='card_id' else x[0] for x in other_months_groupped]

In [17]:
other_months_groupped.head()

,card_id,other_months_payment_from_end_to_begging_,other_months_purchase_amount_max,other_months_purchase_amount_mean,other_months_purchase_amount_min,other_months_purchase_amount_std,other_months_purchase_amount_sum,other_months_authorized_flag_mean,other_months_installments_max,other_months_installments_mean,other_months_installments_min,other_months_installments_std,other_months_installments_sum,other_months_no_installments_mean,other_months_NNN_installments_mean,other_months_no_city_mean,other_months_cat2_dum_0.0_mean,other_months_cat2_dum_1.0_mean,other_months_cat2_dum_2.0_mean,other_months_cat2_dum_3.0_mean,other_months_cat2_dum_4.0_mean,other_months_cat2_dum_5.0_mean,other_months_cat3_dum_A_mean,other_months_cat3_dum_B_mean,other_months_cat3_dum_C_mean,other_months_cat3_dum_nan_mean,other_months_purchase_amount_unusual_mean,other_months_purchase_date_day_diff_in_days_max,other_months_purchase_date_day_diff_in_days_min,other_months_purchase_date_day_diff_in_days_mean,other_months_purchase_date_day_diff_in_days_std
0,C_ID_00007093c1,2,30.044178,24.791385,22.786373,1.913250,297.496623,0.666667,5.0,1.583333,1.0,1.164500,19.0,0.0,0.0,0.166667,0.166667,0.0,0.0,0.833333,0.0,0.000000,0.0,0.666667,0.333333,0.0,0.0,19,0,2.272727,25
1,C_ID_00007093c1,3,26.077179,24.313953,22.906585,1.211807,243.139529,0.800000,1.0,1.000000,1.0,0.000000,10.0,0.0,0.0,0.200000,0.200000,0.0,0.0,0.800000,0.0,0.000000,0.0,1.000000,0.000000,0.0,0.0,22,0,3.222222,29
2,C_ID_00007093c1,4,28.541527,25.016833,22.906585,1.872702,300.201996,0.916667,3.0,1.333333,1.0,0.778499,16.0,0.0,0.0,0.083333,0.083333,0.0,0.0,0.916667,0.0,0.000000,0.0,0.833333,0.166667,0.0,0.0,15,0,2.636364,29
3,C_ID_00007093c1,5,30.449894,24.885702,22.786373,3.014800,149.314210,0.833333,6.0,2.000000,1.0,2.000000,12.0,0.0,0.0,0.500000,0.500000,0.0,0.0,0.333333,0.0,0.166667,0.0,0.666667,0.333333,0.0,0.0,10,0,3.800000,19
4,C_ID_00007093c1,6,24.845005,23.868803,22.786373,0.798078,405.769653,0.705882,2.0,1.058824,1.0,0.242536,18.0,0.0,0.0,0.294118,0.294118,0.0,0.0,0.705882,0.0,0.000000,0.0,0.941176,0.058824,0.0,0.0,10,0,1.687500,27


In [38]:
cols_to_agg = [x for x in other_months_groupped if not re.search('from_end_to',x) and x!='card_id']
cats = {x:'mean' for x in cols_to_agg if re.search('cat|NNN|no|flag|unusual',x)} #categorical columns. Only mean will aggregate
non_std = {x:['min','mean','max','sum'] for x in cols_to_agg if re.search('^((?!std).)*$',x) and x not in cats.keys()}
std_cols = {x:'mean' for x in cols_to_agg if x not in non_std.keys() and x not in cats.keys()}

In [39]:
all_cols_to_agg = {**cats,**non_std,**std_cols}

In [41]:
other_months_all_groupped = other_months_groupped.groupby(['card_id']).agg(all_cols_to_agg)

In [50]:
other_months_all_groupped.columns = [f'{x[0][13:]}_{x[1]}' for x in other_months_all_groupped]

In [51]:
other_months_all_groupped.head()

,authorized_flag_mean_mean,no_installments_mean_mean,NNN_installments_mean_mean,no_city_mean_mean,cat2_dum_0.0_mean_mean,cat2_dum_1.0_mean_mean,cat2_dum_2.0_mean_mean,cat2_dum_3.0_mean_mean,cat2_dum_4.0_mean_mean,cat2_dum_5.0_mean_mean,cat3_dum_A_mean_mean,cat3_dum_B_mean_mean,cat3_dum_C_mean_mean,cat3_dum_nan_mean_mean,purchase_amount_unusual_mean_mean,purchase_amount_max_min,purchase_amount_max_mean,purchase_amount_max_max,purchase_amount_max_sum,purchase_amount_mean_min,purchase_amount_mean_mean,purchase_amount_mean_max,purchase_amount_mean_sum,purchase_amount_min_min,purchase_amount_min_mean,purchase_amount_min_max,purchase_amount_min_sum,purchase_amount_sum_min,purchase_amount_sum_mean,purchase_amount_sum_max,purchase_amount_sum_sum,installments_max_min,installments_max_mean,installments_max_max,installments_max_sum,installments_mean_min,installments_mean_mean,installments_mean_max,installments_mean_sum,installments_min_min,installments_min_mean,installments_min_max,installments_min_sum,installments_sum_min,installments_sum_mean,installments_sum_max,installments_sum_sum,purchase_date_day_diff_in_days_max_min,purchase_date_day_diff_in_days_max_mean,purchase_date_day_diff_in_days_max_max,purchase_date_day_diff_in_days_max_sum,purchase_date_day_diff_in_days_min_min,purchase_date_day_diff_in_days_min_mean,purchase_date_day_diff_in_days_min_max,purchase_date_day_diff_in_days_min_sum,purchase_date_day_diff_in_days_mean_min,purchase_date_day_diff_in_days_mean_mean,purchase_date_day_diff_in_days_mean_max,purchase_date_day_diff_in_days_mean_sum,purchase_amount_std_mean,installments_std_mean,purchase_date_day_diff_in_days_std_mean
card_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
C_ID_00007093c1,0.780535,0.000000,0.0,0.190526,0.190526,0.000000,0.000000,0.796654,0.0,0.012821,0.000000,0.814623,0.185377,0.000000,0.024434,24.845005,29.728390,45.070690,386.469070,23.841023,24.920530,26.663964,323.966884,22.711240,22.958484,24.273997,298.460293,106.655856,284.734584,471.313234,3701.549595,1.0,2.615385,6.0,34.0,1.000000,1.348480,2.000000,17.530237,1.0,1.000000,1.0,13.0,7.0,14.769231,29.0,192.0,2,10.384615,22,135,0,0.000000,0,0,0.181818,2.350735,6.000000,30.559560,2.285816,0.625616,21.230769
C_ID_0001238066,0.986326,0.024887,0.0,0.070405,0.070405,0.815966,0.000000,0.000000,0.0,0.113629,0.000000,0.698874,0.276239,0.024887,0.005495,24.259873,28.508318,37.680952,199.558227,23.486082,24.009906,24.776195,168.069341,22.591028,22.681423,22.725065,158.769963,46.972165,482.567896,895.571226,3377.975274,3.0,4.428571,10.0,31.0,1.176471,1.589704,2.135135,11.127925,0.0,0.714286,1.0,5.0,4.0,31.571429,79.0,221.0,1,4.428571,6,31,0,0.285714,1,2,0.805556,1.246713,1.900000,8.726988,1.577515,1.149694,21.857143
C_ID_0001506ef0,0.944444,0.987179,0.0,0.000000,0.000000,0.008547,0.000000,0.991453,0.0,0.000000,0.987179,0.012821,0.000000,0.000000,0.180098,22.776305,31.613327,44.935451,410.973254,22.655492,25.382511,30.171903,329.972640,22.595085,22.742761,22.981717,295.655896,22.981717,125.690862,429.287817,1633.981207,0.0,0.076923,1.0,1.0,0.000000,0.012821,0.166667,0.166667,0.0,0.000000,0.0,0.0,0.0,0.076923,1.0,1.0,0,7.000000,16,91,0,1.615385,10,21,0.000000,3.282935,10.000000,42.678151,4.788959,0.034021,11.000000
C_ID_0001793786,0.912183,0.986708,0.0,0.005348,0.398827,0.096535,0.453287,0.047022,0.0,0.004329,0.986708,0.013292,0.000000,0.000000,0.228879,40.184519,52.762884,75.541450,580.391725,26.429320,29.688770,35.229264,326.576465,22.545949,23.457361,28.105908,258.030973,64.142960,614.444457,1080.611318,6758.889023,0.0,0.272727,1.0,3.0,0.000000,0.013292,0.058824,0.146211,0.0,0.000000,0.0,0.0,0.0,0.454545,2.0,5.0,0,5.818182,13,64,0,0.000000,0,0,0.000000,0.832478,1.733333,9.157259,8.693669,0.059150,17.909091
C_ID_000183fdda,0.870000,0.018544,0.0,0.028312,0.028312,0.051852,0.005682,0.909346,0.0,0.004808,0.000000,0.659458,0.321998,0.018544,0.108326,25.403991,38.481602,57.647880,307.852815,23.611479,25.479377,29.621933,203.835015,22.621081,23.065453

In [52]:
last_month_groupped.head()

,card_id,last_month_purchase_amount_max,last_month_purchase_amount_mean,last_month_purchase_amount_min,last_month_purchase_amount_std,last_month_purchase_amount_sum,last_month_authorized_flag_mean,last_month_installments_max,last_month_installments_mean,last_month_installments_min,last_month_installments_std,last_month_installments_sum,last_month_no_installments_mean,last_month_NNN_installments_mean,last_month_no_city_mean,last_month_cat2_dum_0.0_mean,last_month_cat2_dum_1.0_mean,last_month_cat2_dum_2.0_mean,last_month_cat2_dum_3.0_mean,last_month_cat2_dum_4.0_mean,last_month_cat2_dum_5.0_mean,last_month_cat3_dum_A_mean,last_month_cat3_dum_B_mean,last_month_cat3_dum_C_mean,last_month_cat3_dum_nan_mean,last_month_purchase_amount_unusual_mean,last_month_purchase_date_day_diff_in_days_max,last_month_purchase_date_day_diff_in_days_min,last_month_purchase_date_day_diff_in_days_mean,last_month_purchase_date_day_diff_in_days_std
0,C_ID_00007093c1,23.432513,23.357380,23.282248,0.106254,46.714760,1.0,1.0,1.000000,1.0,0.000000,2.0,0.000000,0.0,0.000000,0.000000,0.500000,0.0,0.5,0.0,0.000000,0.0,1.000000,0.000000,0.000000,0.0,6,6,6.000000,6
1,C_ID_0001238066,26.287550,24.339129,22.817327,1.103024,219.052161,1.0,10.0,2.555556,1.0,3.244654,23.0,0.000000,0.0,0.111111,0.111111,0.777778,0.0,0.0,0.0,0.111111,0.0,0.777778,0.222222,0.000000,0.0,10,1,3.625000,29
2,C_ID_0001506ef0,22.846479,22.763232,22.679985,0.117729,45.526464,1.0,0.0,0.000000,0.0,0.000000,0.0,1.000000,0.0,0.000000,0.000000,0.000000,0.0,1.0,0.0,0.000000,1.0,0.000000,0.000000,0.000000,0.0,6,6,6.000000,6
3,C_ID_0001793786,45.070690,28.095465,22.621081,7.012091,280.954648,1.0,0.0,0.000000,0.0,0.000000,0.0,1.000000,0.0,0.000000,0.200000,0.200000,0.1,0.5,0.0,0.000000,1.0,0.000000,0.000000,0.000000,0.2,13,0,2.333333,21
4,C_ID_000183fdda,28.923200,25.066796,23.071576,3.340416,75.200389,1.0,1.0,0.666667,0.0,0.577350,2.0,0.333333,0.0,0.000000,0.000000,0.000000,0.0,1.0,0.0,0.000000,0.0,0.666667,0.000000,0.333333,0.0,22,4,13.000000,26


In [54]:
everything_groupped = pd.DataFrame()

In [65]:
for last_month_name in [x for x in last_month_groupped if x!='card_id']:
    for other_month_name in [x for x in other_months_all_groupped if re.search(last_month_name[11:],x)]:
        everything_groupped[f'proc_{other_month_name}'] = last_month_groupped[last_month_name].values / other_months_all_groupped[other_month_name].values

C:\Users\User\Anaconda3\envs\ml\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\User\Anaconda3\envs\ml\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until


In [70]:
everything_groupped['card_id'] = last_month_groupped['card_id']

In [73]:
feather.write_dataframe(everything_groupped.replace({np.inf:np.nan}),'gropped_columns_per_month_to_last')

In [75]:
last_4months_of_activity = all_transactions.groupby(['card_id','payment_from_end_to_begging'])['authorized_flag'].max()

In [83]:
last_4months_of_activity = last_4months_of_activity.to_frame().unstack()

In [81]:
active_months_cat = pd.DataFrame()

In [99]:
last_4months_of_activity.columns = list(range(0,16))

In [111]:
all_month_max_months = all_transactions.groupby(['card_id'])['payment_from_end_to_begging'].max()

In [113]:
last_4months_of_activity['max'] = all_month_max_months + 1

In [125]:
last_4months_of_activity['percent_of_month_activity'] = last_4months_of_activity[list(range(16))].sum(axis=1)/ last_4months_of_activity['max']

In [127]:
last_4months_of_activity['percent_of_month_activity4'] = ((last_4months_of_activity['max'] > 4)*(last_4months_of_activity[list(range(4))].sum(axis=1)/4))+\
                                                         ((last_4months_of_activity['max'] <= 4)*(last_4months_of_activity[list(range(4))].sum(axis=1)/last_4months_of_activity['max']))

In [139]:
categorize = last_4months_of_activity[list(range(4))].copy()

In [147]:
for x in categorize:
    categorize[x] = categorize[x].replace({np.nan:0}).astype(np.int8)

In [157]:
import itertools
bin_perm = {x[1]:x[0] for x in enumerate(["".join(seq) for seq in itertools.product("01", repeat=4)],1)}

In [149]:
categorize['category'] = categorize[0].map(str) + categorize[1].map(str) + categorize[2].map(str) + categorize[3].map(str)

In [158]:
bin_perm

{'0000': 1,
 '0001': 2,
 '0010': 3,
 '0011': 4,
 '0100': 5,
 '0101': 6,
 '0110': 7,
 '0111': 8,
 '1000': 9,
 '1001': 10,
 '1010': 11,
 '1011': 12,
 '1100': 13,
 '1101': 14,
 '1110': 15,
 '1111': 16}

In [161]:
categorize['category'] = categorize['category'].map(bin_perm)

In [164]:
last_4months_of_activity['last4_months_cat'] = categorize['category']

In [165]:
last_4months_of_activity[['percent_of_month_activity','percent_of_month_activity4','last4_months_cat']]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,max,percent_of_month_activity,percent_of_month_activity4,last4_months_cat
card_id,,,,,,,,,,,,,,,,,,,,
C_ID_00007093c1,1.0,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,15,0.933333,0.75,12
C_ID_0001238066,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8,1.000000,1.00,16
C_ID_0001506ef0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,1.0,1.0,1.0,NaN,15,0.933333,1.00,16
C_ID_0001793786,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,12,1.000000,1.00,16
C_ID_000183fdda,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9,0.888889,1.00,16
C_ID_00024e244b,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,1.0,1.0,1.0,1.0,1.0,NaN,NaN,14,0.928571,1.00,16
C_ID_0002709b5a,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,0.0,16,0.812500,1.00,16
C_ID_00027503e2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9,1.000000,1.00,16
C_ID_000298032a,1.0,NaN,1.0,1.0,1.0,1.0,0.0,NaN,1.0,NaN,NaN,1.0,1.0,1.0,NaN,NaN,14,0.642857,0.75,12


In [166]:
feather.write_dataframe(pd.merge(everything_groupped.replace({np.inf:np.nan}),last_4months_of_activity[['percent_of_month_activity','percent_of_month_activity4','last4_months_cat']],how='left',on='card_id')
                        ,'gropped_columns_per_month_to_last_ver2')

In [167]:
gropped_columns_per_month_to_last_ver2 =  feather.read_dataframe('gropped_columns_per_month_to_last_ver2')